In [1]:
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
user_item_matrix = np.array([
    [5,3,0,1],
    [4,0,4,0],
    [0,2,1,4],
    [1,0,5,4],
])

user_similarity = cosine_similarity(user_item_matrix)
def get_user_based_reccommendations(user_id,user_similarity,user_item_matrix,num_reccommendations=3):
    user_similarities = user_similarity[user_id]
    similar_users_sorted = np.argsort(user_similarities[::-1])
    unrated_items = np.where(user_item_matrix[user_id]==0)[0]
    reccommendations = []
    for user in similar_users_sorted:
        for item in unrated_items:
            if user_item_matrix[user,item] > 0:
                reccommendations.append((item, user_item_matrix[user,item]))
    reccommendations.sort(key=lambda x: x[1], reverse = True)
    recommended_items = [item for item, _ in reccommendations[:num_reccommendations]]
    return recommended_items
target_user_id = 0
reccommendations = get_user_based_reccommendations(target_user_id, user_similarity, user_item_matrix)
print("Reccommended items for the target user : ")
for item_id in reccommendations:
    print(f"Item {item_id}")

Reccommended items for the target user : 
Item 2
Item 2
Item 2


<h1> item based coil filter </h1>

In [3]:
item_similarity = cosine_similarity(user_item_matrix.T)
def get_user_based_reccommendations(user_id,item_similarity,user_item_matrix,num_reccommendations=3):
    user_similarities = item_similarity[user_id]
    similar_users_sorted = np.argsort(user_similarities[::-1])
    unrated_items = np.where(user_item_matrix[user_id]==0)[0]
    reccommendations = []
    for user in similar_users_sorted:
        for item in unrated_items:
            if user_item_matrix[user,item] > 0:
                reccommendations.append((item, user_item_matrix[user,item]))
    reccommendations.sort(key=lambda x: x[1], reverse = True)
    recommended_items = [item for item, _ in reccommendations[:num_reccommendations]]
    return recommended_items
target_user_id = 0
reccommendations = get_user_based_reccommendations(target_user_id, item_similarity, user_item_matrix)
print("Reccommended items for the target user : ")
for item_id in reccommendations:
    print(f"Item {item_id}")

Reccommended items for the target user : 
Item 2
Item 2
Item 2


In [4]:
from scipy.sparse.linalg import svds
matrix = np.array(user_item_matrix, float)
u,sigma, Vt = svds(matrix, k=2)
sigma_diag = np.diag(sigma)
predicted_user_item_matrix = np.dot(np.dot(u,sigma_diag),Vt)
def get_model_based_reccommendations(user_id, predicted_user_item_matrix, num_reccommendations=3):
    predicted_interactions = predicted_user_item_matrix[user_id]
    unrated_items = np.where(predicted_interactions == 0)[0]
    sorted_unrated_items = np.argsort(predicted_interactions)[::-1]
    recommended_items = [item for item in sorted_unrated_items[:num_reccommendations]]
    return recommended_items
target_user_id = 0
reccommendations = get_model_based_reccommendations(target_user_id, predicted_user_item_matrix)
print("Reccommended items for the target user : ")
for item_id in reccommendations:
    print(f"Item {item_id}")


Reccommended items for the target user : 
Item 0
Item 1
Item 2


In [5]:
user_similarity

array([[1.        , 0.5976143 , 0.36885556, 0.23473824],
       [0.5976143 , 1.        , 0.15430335, 0.65465367],
       [0.36885556, 0.15430335, 1.        , 0.70710678],
       [0.23473824, 0.65465367, 0.70710678, 1.        ]])

In [6]:
item_similarity

array([[1.        , 0.64194074, 0.5       , 0.24174689],
       [0.64194074, 1.        , 0.0855921 , 0.531085  ],
       [0.5       , 0.0855921 , 1.        , 0.64465837],
       [0.24174689, 0.531085  , 0.64465837, 1.        ]])

In [7]:
predicted_user_item_matrix

array([[ 5.15341682,  1.94976022,  1.07587267, -0.09126383],
       [ 3.7873761 ,  1.47704855,  2.47308024,  1.5491318 ],
       [ 0.32711665,  0.18960034,  2.57880948,  2.4059535 ],
       [ 0.89706489,  0.45748848,  4.69026123,  4.30993496]])

<h1> Recommendation_system_all_Filtering</h1>

In [8]:
import pandas as pd
data = pd.read_csv("D:\_WorkSpace\MyCodes\Machine Learning\Data\\fashion_products.csv")
print(data.head())

   User ID  Product ID Product Name   Brand         Category  Price    Rating  \
0       19           1        Dress  Adidas    Men's Fashion     40  1.043159   
1       97           2        Shoes     H&M  Women's Fashion     82  4.026416   
2       25           3        Dress  Adidas  Women's Fashion     44  3.337938   
3       57           4        Shoes    Zara    Men's Fashion     23  1.049523   
4       79           5      T-shirt  Adidas    Men's Fashion     79  4.302773   

    Color Size  
0   Black   XL  
1   Black    L  
2  Yellow   XL  
3   White    S  
4   Black    M  


In [9]:
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [10]:
content_df = data[['Product ID', 'Product Name', 'Brand', 'Category', 'Color', 'Size']]
content_df['content'] = content_df.apply(lambda row: ''.join(row.dropna().astype(str)), axis=1)

tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(content_df['content'])

content_similarity = linear_kernel(content_matrix, content_matrix)

reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(data[['User ID', 'Product ID', 'Rating']], reader)

C:\Users\hp\AppData\Local\Temp\ipykernel_16540\2407153283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['content'] = content_df.apply(lambda row: ''.join(row.dropna().astype(str)), axis=1)


In [11]:
def get_content_based_recommendations(product_id, top_n):
  index = content_df[content_df['Product ID'] == product_id].index[0]
  similarity_scores = content_similarity[index]
  similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
  recommendations = content_df.loc[similar_indices, 'Product ID'].values
  return recommendations

In [12]:
from surprise.prediction_algorithms import predictions
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
  testset = trainset.build_anti_testset()
  testset = filter(lambda x: x[0] == user_id, testset)
  predictions = algo.test(testset)
  predictions.sort(key=lambda x: x.est, reverse=True)
  recommendations = [prediction.ild for prediction in predictions[:top_n]]
  return recommendations